In [1]:
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Define the neural network class
class Net(nn.Module):
    def __init__(self, inputs, num_layers, num_neurons):
        super(Net, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(inputs, num_neurons))
        for i in range(num_layers - 1):
            self.layers.append(nn.Linear(num_neurons, num_neurons))
        self.output_layer = nn.Linear(num_neurons, 1)

    def forward(self, x, y, t):
        inputs = torch.cat([x, y, t], axis=1)
        out = inputs
        for layer in self.layers:
            out = torch.sin(layer(out))
        output = self.output_layer(out)
        return output

In [5]:
### (2) Model
num_layers = 10
num_neurons = 10

net = Net(3, num_layers, num_neurons)
net = net.to(device)

In [6]:
# Define the loss function and optimizer
mse_cost_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters())

In [51]:
# Definir la función que representa la EDP
def f(x, y, t, c, net):
    # u = net(torch.cat([x, y], axis=1), t)
    u = net(x, y, t)
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    u_y = torch.autograd.grad(u.sum(), y, create_graph=True)[0]
    u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y.sum(), y, create_graph=True)[0]
    u_tt = torch.autograd.grad(u_t.sum(), t, create_graph=True)[0]
    f_val = u_tt - c**2 * (u_xx + u_yy)
    return f_val.sum()

In [52]:
# Define the initial condition
def gaussian(x, y):
    return np.exp(-((x-0.5)**2 + (y-0.5)**2)/0.1)

In [53]:
# Definir la función que representa las condiciones de borde
def boundary_condition(x, y, t, c):
    return torch.tensor([0.0])

In [54]:
# Definir la función para generar datos de entrenamiento
def generate_training_data(n_t, n_x, n_y):
    t = np.random.uniform(low=0.0, high=1.0, size=(n_t, 1))
    x = np.random.uniform(low=0.0, high=2.0, size=(n_x, 1))
    y = np.random.uniform(low=0.0, high=2.0, size=(n_y, 1))
    return t, x, y

In [ ]:
# Definir los parámetros de entrenamiento
num_layers = 10
num_neurons = 10
learning_rate = 0.001
iterations = 20000
n_t = 500
n_x = 100
n_y = 100

c = 1.0

a = 0.0
b = 1.0
t_a = 0.0
t_b = 1.0

In [56]:
# # Set up the boundary conditions
# c = 1.
# n_t = 100
# n_x = 100
# n_y = 100
# x_min, x_max = -5, 5
# y_min, y_max = -5, 5
# t_min, t_max = 0, 1

In [57]:
# Definir la red neuronal
net = Net(inputs=3, num_layers=num_layers, num_neurons=num_neurons).to(device)

In [58]:
# Definir la función de costo y el optimizador
mse_cost_function = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [59]:
# Generar los datos de entrenamiento
t_train, x_train, y_train = generate_training_data(n_t, n_x, n_y)

In [60]:
x_bc = np.random.uniform(low=a, high=b, size=(n_t,1))
y_bc = np.random.uniform(low=a, high=b, size=(n_t,1))
t_bc = np.zeros((n_t,1))
# compute u based on BC
u_bc = 6*np.exp(-3*x_bc)

In [61]:
for epoch in range(iterations):
    optimizer.zero_grad() # to make the gradients zero
    
    # Loss based on boundary conditions
    pt_x_bc = Variable(torch.from_numpy(x_bc).float(), requires_grad=False).to(device)
    pt_y_bc = Variable(torch.from_numpy(y_bc).float(), requires_grad=False).to(device)
    pt_t_bc = Variable(torch.from_numpy(t_bc).float(), requires_grad=False).to(device)
    pt_u_bc = Variable(torch.from_numpy(u_bc).float(), requires_grad=False).to(device)
    
    net_bc_out = net(pt_x_bc, pt_y_bc, pt_t_bc) # output of u(x,t)
    mse_u = mse_cost_function(net_bc_out, pt_u_bc)
    
    # Loss based on PDE
    x_collocation = np.random.uniform(low=a, high=b, size=(500,2))
    y_collocation = np.random.uniform(low=a, high=b, size=(500,2))
    t_collocation = np.random.uniform(low=t_a, high=t_b, size=(500,1))
    all_zeros = np.zeros((500,1))
    
    pt_x_collocation = Variable(torch.from_numpy(x_collocation).float(), requires_grad=True).to(device)
    pt_y_collocation = Variable(torch.from_numpy(y_collocation).float(), requires_grad=True).to(device)
    pt_t_collocation = Variable(torch.from_numpy(t_collocation).float(), requires_grad=True).to(device)
    pt_all_zeros = Variable(torch.from_numpy(all_zeros).float(), requires_grad=False).to(device)
    
    f_out = f(pt_x_collocation[:,0:1], pt_y_collocation[:,0:1], pt_t_collocation, c, net) # output of f(x,t)
    mse_f = mse_cost_function(f_out, pt_all_zeros)
    
    # Combining the loss functions
    loss = mse_u + mse_f
    
    loss.backward() # This is for computing gradients using backward propagation
    optimizer.step() # This is equivalent to : theta_new = theta_old - alpha * derivative of J w.r.t theta

    with torch.autograd.no_grad():  
        if epoch % 1000 == 0:
            print(epoch, "Training Loss:", loss.data)


c:\Users\Det-Pc\anaconda3\envs\pytorch-C-DL\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([500, 1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0 Training Loss: tensor(4.3678, device='cuda:0')
1000 Training Loss: tensor(1.7776, device='cuda:0')
2000 Training Loss: tensor(2.0359, device='cuda:0')
3000 Training Loss: tensor(0.8104, device='cuda:0')
4000 Training Loss: tensor(0.7641, device='cuda:0')
5000 Training Loss: tensor(0.6026, device='cuda:0')
6000 Training Loss: tensor(0.5678, device='cuda:0')
7000 Training Loss: tensor(0.9221, device='cuda:0')
8000 Training Loss: tensor(0.8776, device='cuda:0')
9000 Training Loss: tensor(0.5199, device='cuda:0')
10000 Training Loss: tensor(0.5101, device='cuda:0')
11000 Training Loss: tensor(0.7546, device='cuda:0')
12000 Training Loss: tensor(0.4329, device='cuda:0')
13000 Training Loss: tensor(1.2174, device='cuda:0')
14000 Training Loss: tensor(0.3318, device='cuda:0')
15000 Training Loss: tensor(0.3077, device='cuda:0')
16000 Training Loss: tensor(0.3147, device='cuda:0')
17000 Training Loss: tensor(0.2969, device='cuda:0')
18000 Training Loss: tensor(0.3704, device='cuda:0')
19000 